Распознование с опечатками


In [49]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import pymorphy2
from nltk.stem import SnowballStemmer


from nltk.tokenize import word_tokenize

#Create lemmatizer and stopwords list
mystem = Mystem() 
from spellchecker import SpellChecker
spell = SpellChecker(language='ru')
russian_stopwords = stopwords.words("russian")



#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [ token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    text = word_tokenize(text)
    correct=[]
    misspelled = spell.unknown(text)
    for word in misspelled:
        correct.append(spell.correction(word))
        
    

    return correct

preprocess_text('херург серце')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


['хирург', 'сердце']

Определение по симптомам 

In [37]:
def check_doctor(simptomy):
    zapis=0
    cardio = open('cardio.txt', 'r')
    gastro = open('gastroenterologist.txt','r')
    theraphist = open('therapist.txt','r')
    surgeon = open('surgeon.txt', 'r')
    spisok_simptomov_cardio=preprocess_text(cardio.read())
    spisok_simptomov_gastro=preprocess_text(gastro.read())
    spisok_simptomov_surgeon=preprocess_text(surgeon.read())
    spisok_simptomov_theraphist=preprocess_text(theraphist.read())
    simptomy=preprocess_text(simptomy)
    result_cardio=set(spisok_simptomov_cardio) & set(simptomy)
    result_surgeon=set(spisok_simptomov_surgeon) & set(simptomy)
    result_gastro=set(spisok_simptomov_gastro) & set(simptomy)
    if result_cardio:
      zapis = 1
    elif result_gastro:
      zapis = 2
    elif result_surgeon:
      zapis = 3
    else:
      zapis = 4
  
    cardio.close()
    gastro.close()
    surgeon.close()
    theraphist.close()
    return(zapis)
check_doctor('у меня понос')

2

Определение по запросу и времени и имени врача

In [109]:
import re
from nltk.stem import SnowballStemmer
from spellchecker import SpellChecker
spell = SpellChecker(language='ru')
def date_time_zapis(message):
    snowball = SnowballStemmer(language="russian")
    date = re.search(r'\d\d\.\d\d\.\d{4}',message) 
    date = (date[0] if date else 0) 
    suggest = 0;
    time_check = re.search(r'([01]?[0-9]|2[0-3]):([0-5][0-9])',message) 
    time_check = (1 if message else 0) 
    check_zapis = re.search(r'записываться', message)
    check_zapis = (1 if message else 0)
    if check_zapis == 1 and time_check == 0:
      suggest =1
    if check_zapis == 1 and time_check == 1:
      time = re.search(r'([01]?[0-9]|2[0-3]):([0-5][0-9])',message) 
      time = (time[0] if message else 0) 
      spisok_doctors = open('spisok_doctors.txt', 'r')
      spisok_doctors = word_tokenize(spisok_doctors.read())
      for (index, elem) in enumerate(spisok_doctors):
        spisok_doctors[index] = snowball.stem(elem)
      # message = preprocess_text(message)
      message = word_tokenize(message)
      for (index, elem) in enumerate(message):
        message[index] = snowball.stem(elem)
      correct = message
      # misspelled = spell.unknown(message)
      # for word in misspelled:
      #   if word =="айболит":
      #     correct.append('айбол')
      #   if word =='пузиков':
      #     correct.append('пузиков')
      #   if word =='резник':
      #     correct.append('резник')
      #   if word =='ишемит':
      #     correct.append('ишемит')
      #   correct.append(spell.correction(word))
      therapist_name = spisok_doctors[0:4]
      surgeon_name = spisok_doctors[4:8]
      gastro_name = spisok_doctors[8:12]
      cardiolog_name = spisok_doctors[12:16]
      name_cardio=(set(cardiolog_name) & set(correct))
      name_gastro = set(gastro_name) & set(correct)
      name_surgeon = (set(surgeon_name) & set(correct))
      nane_therapist =(set(therapist_name) & set(correct))
      print(cardiolog_name)
      print(correct)
      if any([bool(set(cardiolog_name) & set(correct)) and bool(set(correct) & set(gastro_name)),
      bool(set(therapist_name) & set(correct)) and bool(set(cardiolog_name) & set(correct)) ,
      bool(set(surgeon_name) & set(correct)) and bool(set(cardiolog_name) & set(correct)) ,
      bool(set(therapist_name) & set(correct)) and bool(set(gastro_name) & set(correct)),
      bool(set(therapist_name) & set(correct)) and bool(set(surgeon_name) & set(correct)),
      bool(set(gastro_name) & set(correct)) and bool(set(cardiolog_name) & set(correct)) ]
      ):
        doctor = 0
      elif name_cardio:
        doctor = 1
      elif name_gastro:
        doctor = 2
      elif name_surgeon:
        doctor = 3
      else:
        doctor = 4
      # print(spisok_doctors)
      # print(check_zapis)
      # print(time)
    return(time,date,doctor,suggest)    
date_time_zapis('Я хотел бы запишите к марату ренатовичу на 19:00, 21.02.2001')

['кардиолог', 'ишемит', 'марат', 'ренатович']
['я', 'хотел', 'бы', 'запиш', 'к', 'марат', 'ренатович', 'на', '19:00', ',', '21.02.2001']


('19:00', '21.02.2001', 1, 0)

проверка даты

In [12]:
import re
import time
from datetime import datetime
def check_date(date):
    try:
      correct = time.strptime(date, '%m.%d.%Y')
    except ValueError:
      correct = 0
    date = re.fullmatch(r'\d\d\.\d\d\.\d{4}',date) 
    date = (date[0] if date else 0) 
    current_date = datetime.now()
    date_now = [current_date.year,current_date.month,current_date.day]
    date=[int(date.split('.')[2]),int(date.split('.')[1]),int(date.split('.')[0])]
    if date[0] < date_now[0]-18 and correct !=0: 
      date = 1
    else:
     date = 0
    return date
check_date('11.12.2003')

1

In [57]:
import re
from nltk.stem import SnowballStemmer
time = re.search(r'([01]?[0-9]|2[0-3]):([0-5][0-9])',r'Я хотел бы запишите на 9:00') 
time = (time[0] if time else 0) 
check_zapis = re.search(r'записываться',r'Я хотел бы запишите на 9:00' )
check_zapis = (1 if check_zapis else 0)
spisok_doctors = open('spisok_doctors.txt', 'r')
spisok_doctors = word_tokenize(spisok_doctors.read())
snowball = SnowballStemmer(language="russian")
for (index, elem) in enumerate(spisok_doctors):
    spisok_doctors[index] = snowball.stem(elem)
message = preprocess_text('Я хотел бы запишите айболиту на 9:00')
therapist_name = spisok_doctors[0:4]
surgeon_name = spisok_doctors[4:8]
gastro_name = spisok_doctors[8:12]
cardiolog_name = spisok_doctors[12:16]
doctor = 0
name_cardio=set(cardiolog_name) & set(message)
if name_cardio != [ ]:
     doctor = 1
name_surgeon=(set(surgeon_name) & set(message))
if name_surgeon != [ ]:
      doctor = 3
name_gastro=(set(gastro_name) & set(message))
if name_gastro:
    doctor = 2
else:
    doctor=4
spisok_doctors


['терапевт',
 'айбол',
 'серг',
 'сергеевич',
 'хирург',
 'резник',
 'макс',
 'владимирович',
 'гастроэнтеролог',
 'пузиков',
 'лид',
 'васильевн',
 'кардиолог',
 'ишемит',
 'марат',
 'ренатович']